# Asignación 3-dimensional  
**Pablo García Pérez**  

En este cuaderno Jupyter, abordaremos el **Problema de Asignación** y exploraremos sus variantes tridimensionales, específicamente las versiones **planar** y **axial**. Además, se proporcionarán algunos supuestos que ayudarán a contextualizar y comprender estas variantes, siendo estos uno para cada uno de los problemas.

---

## Problema 3-dimensional Planar  

### **Modelo**  
Una compañía de transporte de mercancías, *"Logística La Orotava"*, enfrenta un nuevo desafío operativo. Debido a un incremento en la demanda y a cambios en las rutas, los planes logísticos existentes se volvieron ineficientes, generando demoras en las entregas y costos adicionales. Por este motivo, el gerente de la empresa solicitó nuestra ayuda para optimizar la asignación de recursos.

Durante una reunión, el gerente nos proporcionó información clave sobre su operación, enfocada en la distribución de mercancías entre varios centros de distribución y clientes finales. Entre los datos más relevantes se encontraban los siguientes:

- El número de clientes que requieren entregas.  
- La cantidad de conductores disponibles en la plantilla.  
- El número de vehículos de transporte activos.  
- El costo asociado al envío de un paquete desde un centro de distribución hasta el cliente final, que depende del conductor y el vehículo asignado.

---

### **Representación Matemática**  
A partir de esta situación, se modelaron los siguientes conjuntos y parámetros:

- **Clientes:** Representados por el conjunto \( num_clientes = \{1, ..., n\} \).  
- **Conductores:** Representados por el conjunto \( numero_empleados = \{1, ..., m\} \).  
- **Vehículos:** Representados por el conjunto \( número_vehiculos = \{1, ..., h\} \).  
- **Costo de envío:** Denotado como \( \text{Coste}_{d,c,v} \), que indica el costo de que un conductor \( e \) utilice el vehículo \( m \) para entregar mercancías al cliente \( c \).

---

### **Resolvemos el modelo**  



In [ ]:
# Importar los paquetes necesarios
using Pkg
Pkg.add("JuMP")  # Paquete para modelar problemas de optimización
Pkg.add("GLPK")  # Paquete para utilizar el solver GLPK

using JuMP, GLPK, Random

# Definir parámetros iniciales
num_clientes = 4            # Cantidad de clientes
num_empleados = num_clientes # Cantidad de empleados (igual al número de clientes)
num_vehiculos = num_empleados # Cantidad de máquinas (igual al número de empleados)

# Semilla para reproducibilidad
Random.seed!(2160)

# Matriz de costos: coste de asignar un empleado y máquina a cada cliente
costos = rand(20:60, num_clientes, num_empleados, num_vehiculos)

# Crear el modelo de optimización
modelo = Model(GLPK.Optimizer)
set_silent(modelo)  # Suprimir la salida del solver

# Definir variables binarias (1 si el empleado usa la máquina para el cliente, 0 si no)
@variable(modelo, asignacion[1:num_clientes, 1:num_empleados, 1:num_vehiculos], Bin)

# Función objetivo: minimizar el coste total
@objective(modelo, Min, sum(costos[c, e, m] * asignacion[c, e, m] 
                            for c in 1:num_clientes, e in 1:num_empleados, m in 1:num_vehiculos))

# Restricciones:
# Cada máquina puede asignarse solo a un cliente y un empleado
@constraint(modelo, [m=1:num_vehiculos], sum(asignacion[:, :, m]) == 1)

# Cada cliente debe ser atendido por exactamente un empleado y una máquina
@constraint(modelo, [c=1:num_clientes], sum(asignacion[c, :, :]) == 1)

# Cada empleado puede manejar una máquina para un cliente
@constraint(modelo, [e=1:num_empleados], sum(asignacion[:, e, :]) == 1)

# Resolver el modelo
optimize!(modelo)

# Mostrar los resultados
if termination_status(modelo) == MOI.OPTIMAL
    println("Costo total: ", objective_value(modelo), " €.")
    println()
    for c in 1:num_clientes
        for e in 1:num_empleados
            for m in 1:num_vehiculos
                # Comprobar si la asignación es válida
                if value(asignacion[c, e, m]) > 0.5
                    println("El empleado $e utiliza la máquina $m para atender al cliente $c. Coste: ", 
                            costos[c, e, m], " €.")
                end
            end
        end
    end
else
    println("No se encontró una solución óptima para el problema.")
end


    Updating registry at `C:\Users\pablo\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\pablo\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\pablo\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\pablo\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\pablo\.julia\environments\v1.11\Manifest.toml`


Costo total: 106.0 €.

El empleado 4 utiliza la máquina 3 para atender al cliente 1. Coste: 31 €.
El empleado 1 utiliza la máquina 4 para atender al cliente 2. Coste: 30 €.
El empleado 3 utiliza la máquina 1 para atender al cliente 3. Coste: 20 €.
El empleado 2 utiliza la máquina 2 para atender al cliente 4. Coste: 25 €.


### **Resolvemos el modelo**
Definimos cada parte en diferentes secciones para poder explicar en detalle lo que se está haciendo en cada sección del código.

### **1. Importamos las librerías necesarias para que pueda funcionar el código**  
Estas instrucciones configuran el entorno para modelar y resolver problemas de optimización matemática utilizando **JuMP** como herramienta de modelado y **GLPK** como solucionador. Además, incluyen funciones para manejar valores aleatorios con **Random**.


In [ ]:
# Importar los paquetes necesarios
using Pkg
Pkg.add("JuMP")  # Paquete para modelar problemas de optimización
Pkg.add("GLPK")  # Paquete para utilizar el solver GLPK

using JuMP, GLPK, Random

### **2. Introducimos los datos**  
Estas líneas establecen los parámetros y generan una matriz de costos que representa el problema a resolver. Cada elemento de la matriz `costos[c, e, m]` indica el costo asociado a que el empleado \( e \) use el vehículo \( m \) para atender al cliente \( c \). Estos datos se utilizarán en el modelo de optimización para **minimizar los costos totales**.

In [ ]:
# Definir parámetros iniciales
num_clientes = 4            # Cantidad de clientes
num_empleados = num_clientes # Cantidad de empleados (igual al número de clientes)
num_vehiculos = num_empleados # Cantidad de vehículos (igual al número de empleados)

# Semilla para reproducibilidad
Random.seed!(2160)

# Matriz de costos: coste de asignar un empleado y un vehiculo a cada cliente
costos = rand(20:60, num_clientes, num_empleados, num_vehiculos)

### **3. Definimos el modelo** 
Estas líneas modelan el problema de manera que:  
1. **Minimiza los costos totales.**  
2. **Asegura que cada cliente sea atendido.**  
3. **Respeta las limitaciones de asignación de empleados y vehículos.**

El modelo encuentra la mejor combinación de empleados y vehículos para atender a los clientes con el menor costo posible.

In [ ]:
# Crear el modelo de optimización
modelo = Model(GLPK.Optimizer)
set_silent(modelo)  # Suprimir la salida del solver

# Definir variables binarias (1 si el empleado usa el vehículo para el cliente, 0 si no)
@variable(modelo, asignacion[1:num_clientes, 1:num_empleados, 1:num_vehiculos], Bin)

# Función objetivo: minimizar el coste total
@objective(modelo, Min, sum(costos[c, e, m] * asignacion[c, e, m] 
                            for c in 1:num_clientes, e in 1:num_empleados, m in 1:num_vehiculos))
# Restricciones:
# Cada vehñiculo puede asignarse solo a un cliente y un empleado
@constraint(modelo, [m=1:num_vehiculos], sum(asignacion[:, :, m]) == 1)
# Cada cliente debe ser atendido por exactamente un empleado y un vehículo
@constraint(modelo, [c=1:num_clientes], sum(asignacion[c, :, :]) == 1)
# Cada empleado puede manejar una máquina para un cliente
@constraint(modelo, [e=1:num_empleados], sum(asignacion[:, e, :]) == 1)

# Resolver el modelo
optimize!(modelo)

## Problema 3-dimensional Planar 

### **Modelo**  

La empresa **Optimizaciones Tareas Eficientes** tiene una serie de problemas relacionados con la productividad de sus trabajadores. Esto se debe a que la productividad estaba disminuyendo debido a una mala asignación de tareas en sus turnos de trabajo.  

Después de realizar una investigación exhaustiva en la empresa, se descubrió que las tareas no estaban siendo distribuidas adecuadamente entre los turnos y los trabajadores, lo que entorpece gravemente la realización de las tareas.  

Con esta información, se pudo recopilar datos clave y se nos proporcionó la siguiente información:  

- El número total de trabajadores y los turnos que cada uno puede cubrir.  
- La capacidad máxima de tareas que cada turno de cada trabajador puede manejar.  
- La prioridad asignada a cada tarea cuando se realiza en un turno específico por un trabajador en particular.  

## Representación matemática:

1. **Conjunto de trabajadores**, que llamaremos \( W={1...n} \).  
2. **Conjunto de turnos**, que llamaremos \( S={1...m} \).  
3. **Conjunto de tareas**, que llamaremos \( T={1...k} \).  
4. **Capacidad de cada turno de cada trabajador**, que denotaremos como \( C \). Cada posición \( C[w, s] \) indica la capacidad máxima de tareas que el turno \( s \) del trabajador \( w \) puede manejar.  
5. **Prioridad asignada a cada tarea en cada turno de cada trabajador**, que denotaremos como \( P \). Cada posición \( P[w, s, t] \) indica la prioridad asociada a que el trabajador \( w \) realice la tarea \( t \) durante el turno \( s \).  

## Restricciones adicionales:

1. Es necesario que todas las tareas sean asignadas a un trabajador y a un turno.  
2. No es posible asignar más tareas a un turno de las que permite la capacidad del turno correspondiente.  
3. Cada turno de cada trabajador debe tener al menos una tarea asignada.  

## Resolvemos el modelo:


In [3]:
# Importar los paquetes necesarios
using Pkg
Pkg.add("JuMP")  # Paquete para modelar problemas de optimización matemática en Julia.
Pkg.add("GLPK")  # GLPK es un solver para problemas de optimización lineal y entera.

using JuMP, GLPK, Random

# Definir parámetros del problema
num_workers = 4      # Número de trabajadores.
num_shifts = 6      # Número de turnos disponibles por trabajador.
num_tasks = 24       # Número total de tareas.
Random.seed!(1234)    # Establecer la semilla para la aleatoriedad, de modo que los resultados sean reproducibles.

# Generar valores aleatorios para la prioridad (P) y la capacidad (C)
priority = rand(1:10, num_workers, num_shifts, num_tasks)  # Prioridad asociada a cada tarea para cada trabajador en cada turno.
capacity = rand(1:2, num_workers, num_shifts)              # Capacidad de cada turno para cada trabajador.

# Crear un modelo de optimización con el solver GLPK
optimization_model = Model(GLPK.Optimizer)
set_silent(optimization_model)  # Configurar el modelo para que no muestre mensajes durante la optimización.

# Definir variables de decisión
# `assign[w, s, t]` es una variable binaria que indica si la tarea `t` está asignada al turno `s` del trabajador `w`.
@variable(optimization_model, assign[1:num_workers, 1:num_shifts, 1:num_tasks] >= 0, binary=true)

# Definir la función objetivo
# Maximizar la prioridad total por la asignación de tareas.
@objective(optimization_model, Max, sum(priority[w, s, t] * assign[w, s, t] for w in 1:num_workers, s in 1:num_shifts, t in 1:num_tasks))

# Definir restricciones
# Restricción 1: Cada tarea debe ser asignada a un único turno y trabajador.
@constraint(optimization_model, task_assignment[t=1:num_tasks], sum(assign[:, :, t]) == 1)

# Restricción 2: Cada turno de cada trabajador debe tener al menos una tarea asignada.
@constraint(optimization_model, worker_shift[w=1:num_workers, s=1:num_shifts], sum(assign[w, s, :]) >= 1)

# Restricción 3: La cantidad de tareas en un turno no debe exceder la capacidad de ese turno.
@constraint(optimization_model, shift_capacity[w=1:num_workers, s=1:num_shifts], sum(assign[w, s, :]) <= capacity[w, s])

# Optimizar el modelo
optimize!(optimization_model)

# Obtener y mostrar los resultados
max_priority = objective_value(optimization_model)
println("Prioridad máxima alcanzada: ", max_priority)
println()
println("Trabajadores y turnos:")

# Mostrar la asignación de tareas a los turnos y trabajadores
for w in 1:num_workers
    println("  Trabajador $w")
    for s in 1:num_shifts
        shift_capacity_value = capacity[w, s]  # Capacidad del turno `s` del trabajador `w`.
        println("    Turno $s con capacidad para $shift_capacity_value tareas. Contiene las tareas:")
        task_found = false  # Bandera para verificar si hay tareas en este turno.
        for t in 1:num_tasks
            if value(assign[w, s, t]) > 0  # Si la tarea `t` está asignada a este turno.
                task_found = true
                println("      * Tarea $t")
            end
        end
    end
    println()
end


   Resolving package versions...
  No Changes to `C:\Users\pablo\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\pablo\.julia\environments\v1.11\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\pablo\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\pablo\.julia\environments\v1.11\Manifest.toml`


Prioridad máxima alcanzada: 236.0

Trabajadores y turnos:
  Trabajador 1
    Turno 1 con capacidad para 2 tareas. Contiene las tareas:
      * Tarea 10
    Turno 2 con capacidad para 1 tareas. Contiene las tareas:
      * Tarea 14
    Turno 3 con capacidad para 2 tareas. Contiene las tareas:
      * Tarea 3
    Turno 4 con capacidad para 2 tareas. Contiene las tareas:
      * Tarea 24
    Turno 5 con capacidad para 1 tareas. Contiene las tareas:
      * Tarea 13
    Turno 6 con capacidad para 1 tareas. Contiene las tareas:
      * Tarea 22

  Trabajador 2
    Turno 1 con capacidad para 2 tareas. Contiene las tareas:
      * Tarea 7
    Turno 2 con capacidad para 1 tareas. Contiene las tareas:
      * Tarea 6
    Turno 3 con capacidad para 2 tareas. Contiene las tareas:
      * Tarea 2
    Turno 4 con capacidad para 2 tareas. Contiene las tareas:
      * Tarea 8
    Turno 5 con capacidad para 1 tareas. Contiene las tareas:
      * Tarea 18
    Turno 6 con capacidad para 1 tareas. Contie

### **1. Importamos las librerías necesarias para que pueda funcionar el código**  
Estas instrucciones configuran el entorno para modelar y resolver problemas de optimización matemática utilizando **JuMP** como herramienta de modelado y **GLPK** como solucionador. Además, incluyen funciones para manejar valores aleatorios con **Random**.

In [ ]:
# Importar los paquetes necesarios
using Pkg
Pkg.add("JuMP")  # Paquete para modelar problemas de optimización matemática en Julia.
Pkg.add("GLPK")  # GLPK es un solver para problemas de optimización lineal y entera.

using JuMP, GLPK, Random

### **2. Introducimos los datos**  
Estas líneas establecen los parámetros y generan matrices que representan el problema a resolver:

priority indica la prioridad asignada a cada tarea cuando se realiza en un turno específico por un trabajador en particular.
capacity define el número máximo de tareas que cada turno puede manejar.
Estos datos se utilizarán en el modelo de optimización para maximizar la prioridad total de las tareas asignadas.

In [ ]:
# Definir parámetros del problema
num_workers = 4      # Número de trabajadores.
num_shifts = 6      # Número de turnos disponibles por trabajador.
num_tasks = 24       # Número total de tareas.
Random.seed!(1234)    # Establecer la semilla para la aleatoriedad, de modo que los resultados sean reproducibles.

# Generar valores aleatorios para la prioridad (P) y la capacidad (C)
priority = rand(1:10, num_workers, num_shifts, num_tasks)  # Prioridad asociada a cada tarea para cada trabajador en cada turno.
capacity = rand(1:2, num_workers, num_shifts)              # Capacidad de cada turno para cada trabajador.

### **3. Definimos el modelo** 
Estas líneas modelan el problema de manera que:

Maximizan la prioridad total: La función objetivo busca la asignación que genere la mayor prioridad acumulada.
Aseguran que cada tarea sea asignada a un único turno y trabajador: Esto se garantiza mediante restricciones de asignación.
Respetan las limitaciones de capacidad y asignación: Cada turno respeta su capacidad y recibe al menos una tarea.
El modelo encuentra la mejor combinación de trabajadores y turnos para asignar las tareas maximizando la prioridad total.

In [ ]:
# Crear un modelo de optimización con el solver GLPK
optimization_model = Model(GLPK.Optimizer)
set_silent(optimization_model)  # Configurar el modelo para que no muestre mensajes durante la optimización.

# Definir variables de decisión
@variable(optimization_model, assign[1:num_workers, 1:num_shifts, 1:num_tasks] >= 0, binary=true)

# Definir la función objetivo
@objective(optimization_model, Max, sum(priority[w, s, t] * assign[w, s, t] for w in 1:num_workers, s in 1:num_shifts, t in 1:num_tasks))

# Definir restricciones
@constraint(optimization_model, task_assignment[t=1:num_tasks], sum(assign[:, :, t]) == 1)
@constraint(optimization_model, shift_capacity[w=1:num_workers, s=1:num_shifts], sum(assign[w, s, :]) <= capacity[w, s])
@constraint(optimization_model, worker_shift[w=1:num_workers, s=1:num_shifts], sum(assign[w, s, :]) >= 1)

# Optimizar el modelo
optimize!(optimization_model)